IMPORT LIBRARY/PERSIAPAN

In [12]:
import cv2
import os
import pickle
import time
import numpy as np
from PIL import Image as Img
from numpy import asarray, expand_dims
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- KONFIGURASI GLOBAL ---
DB_FILE = "datafacenet_aug.pkl"

# --- LOAD MODEL ---
# Cek agar tidak meload ulang jika sudah ada di memori
if 'MyFaceNet' not in globals():
    print("⏳ Sedang memuat Model FaceNet & MTCNN... (Mohon tunggu)")
    MyFaceNet = FaceNet()
    detector = MTCNN()
    print("✅ Model SIAP digunakan.")
else:
    print("✅ Model sudah aktif.")

# --- KONFIGURASI AUGMENTASI ---
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest'
)

# --- FUNGSI BANTUAN DATABASE ---
def load_db():
    if os.path.exists(DB_FILE):
        try:
            with open(DB_FILE, "rb") as f: return pickle.load(f)
        except: return {}
    return {}

def save_db(database):
    with open(DB_FILE, "wb") as f: pickle.dump(database, f)
    print(f"💾 Database tersimpan. Total: {len(database)} vektor.")

✅ Model sudah aktif.


TAMBAHKAN DATA/INPUT DATA

In [19]:
# Konfigurasi Input
TARGET_FOTO = 20        
INTERVAL = 0.5  # Detik

# Input Data Diri
nik = input("Masukkan NIK : ").strip()
nama = input("Masukkan Nama: ").strip().replace(" ", "_")

if nik and nama:
    database = load_db()
    cap = cv2.VideoCapture(0)
    print(f"\n🎥 KAMERA AKTIF: {nama}")
    print("   Gerakkan wajah sedikit (kiri/kanan/senyum).")
    print("   Tekan [Q] di jendela kamera untuk batal.\n")
    
    foto_count = 0
    last_time = time.time()
    
    while foto_count < TARGET_FOTO:
        ret, frame = cap.read()
        if not ret: break
        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = detector.detect_faces(rgb)
        
        curr_time = time.time()
        
        if res:
            x, y, w, h = res[0]['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Ambil foto setiap 0.5 detik
            if curr_time - last_time > INTERVAL:
                try:
                    # Crop & Proses
                    face = rgb[abs(y):y+h, abs(x):x+w]
                    face_pil = Img.fromarray(face).resize((160, 160))
                    face_inp = expand_dims(asarray(face_pil), axis=0)
                    
                    # 1. Simpan Asli
                    key = f"{nik}_{nama}_{foto_count+1:02d}"
                    database[key] = MyFaceNet.embeddings(face_inp)
                    
                    # 2. Simpan Augmentasi (3 Variasi)
                    gen = datagen.flow(face_inp, batch_size=1)
                    for i in range(3):
                        database[f"{key}_aug{i}"] = MyFaceNet.embeddings(next(gen))
                    
                    foto_count += 1
                    last_time = curr_time
                    print(f"✅ Foto {foto_count}/{TARGET_FOTO} tersimpan (+Augmentasi)")
                    
                    # Indikator Visual (Kotak Merah sesaat)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
                    
                except Exception as e:
                    print(f"Error: {e}")

        # Tampilkan Info
        cv2.putText(frame, f"{foto_count}/{TARGET_FOTO}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        cv2.imshow('Input Data', frame)
        if cv2.waitKey(1) == ord('q'): break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if foto_count > 0:
        save_db(database)
        print("\n🎉 Registrasi Selesai.")
else:
    print("❌ NIK dan Nama wajib diisi!")

❌ NIK dan Nama wajib diisi!


CEK DATABASE

In [14]:
db = load_db()
print(f"\n📊 TOTAL VEKTOR: {len(db)}")
print("-" * 40)
print(f"{'NAMA (NIK)':<30} | {'JUMLAH'}")
print("-" * 40)

rekap = {}
for k in db:
    # Ambil NIK_Nama saja
    parts = k.split('_')
    if len(parts) >= 2:
        id_ = f"{parts[1]} ({parts[0]})"
    else:
        id_ = "Format Salah"
    rekap[id_] = rekap.get(id_, 0) + 1

for nama, jumlah in rekap.items():
    print(f"{nama:<30} | {jumlah}")
print("-" * 40)


📊 TOTAL VEKTOR: 13022
----------------------------------------
NAMA (NIK)                     | JUMLAH
----------------------------------------
Debora (5221911012)            | 240
Anggun (5221911025)            | 240
MichaelAndrewDeHaan (5231811002) | 240
Hamdanu Fahmi Utomo (5231811004) | 234
Akhmad Nabil Saputra (5231811005) | 240
Daniel Granesa Kiara (5231811006) | 240
Amalia Dwi Ramadhani (5231811007) | 240
Sophia (5231811008)            | 234
Otniel Chresto Purwandi (5231811009) | 240
MEYLAN ARYANI (5231811010)     | 240
KusumaRatih (5231811013)       | 240
Dian Eka Pratiwi (5231811014)  | 240
Fadilah Ratu Azzahra (5231811015) | 240
Kesya (5231811016)             | 240
Maulana Ahmad Muhaimin (5231811017) | 240
Sulis Septiani Putri (5231811018) | 240
Chronika (5231811019)          | 240
NASHA SHINTA ABP (5231811021)  | 240
Lathif Ramadhan (5231811022)   | 240
Rahma Fieka Januarni (5231811023) | 240
Maria Febronia Boa (5231811024) | 240
Novera (5231811025)            | 240
ULFAH N

HAPUS DATA

In [17]:
cari = input("Masukkan NIK atau Nama yang mau DIHAPUS: ").strip()

if cari:
    db = load_db()
    # Cari semua key yang mengandung kata kunci
    hapus_list = [k for k in db if cari.lower() in k.lower()]
    
    if hapus_list:
        print(f"\n⚠️ Ditemukan {len(hapus_list)} data (termasuk augmentasi) untuk '{cari}'.")
        print(f"   Contoh: {hapus_list[:2]}...")
        
        yakin = input("🔥 Yakin hapus? (y/n): ").lower()
        if yakin == 'y':
            for k in hapus_list: del db[k]
            save_db(db)
            print("✅ Data berhasil dihapus.")
        else:
            print("🚫 Batal menghapus.")
    else:
        print("❌ Data tidak ditemukan.")


⚠️ Ditemukan 80 data (termasuk augmentasi) untuk 'opung'.
   Contoh: ['6754_opung_01', '6754_opung_01_aug0']...
💾 Database tersimpan. Total: 12942 vektor.
✅ Data berhasil dihapus.


PRESENSI REAL TIME

In [18]:
def mulai_presensi_realtime():
    database = load_database()
    if not database: 
        print("Database kosong!"); return
    
    cap = cv2.VideoCapture(0)
    print("\n🎥 PRESENSI MULAI. Tekan 'q' untuk keluar.")
    
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        # Optimasi: Resize kecil untuk deteksi
        small = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        rgb_small = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        results = detector.detect_faces(rgb_small)
        
        for res in results:
            x, y, w, h = res['box']
            x, y, w, h = x*2, y*2, w*2, h*2 # Balikin ke ukuran asli
            
            x2, y2 = x+w, y+h
            face = frame[abs(y):y2, abs(x):x2]
            
            if face.size > 0:
                try:
                    face_inp = expand_dims(asarray(Img.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB)).resize((160,160))), axis=0)
                    emb = MyFaceNet.embeddings(face_inp)
                    
                    # Identifikasi
                    min_dist = 100
                    identity = "Unknown"
                    for key, db_emb in database.items():
                        dist = np.linalg.norm(emb - db_emb)
                        if dist < min_dist: min_dist, identity = dist, key
                    
                    if min_dist < 0.7:
                        parts = identity.split('_')
                        name_display = f"{parts[1]} ({parts[0]})" if len(parts)>1 else identity
                        color = (0, 255, 0)
                    else:
                        name_display = "Tidak Dikenal"
                        color = (0, 0, 255)
                    
                    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                    cv2.putText(frame, name_display, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                except: pass
        
        cv2.imshow('Presensi Real-Time', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break
            
    cap.release()
    cv2.destroyAllWindows()

# Jalankan
mulai_presensi_realtime()


🎥 PRESENSI MULAI. Tekan 'q' untuk keluar.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━